# Data analysis

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

data = pd.read_csv("douban_data/data.csv")

In [5]:
data.head()

,actor,date,director,id,language,rate,rating_num,region,runtime,title,...,is_historical,is_war,is_western,is_fantasy,is_adventure,is_disaster,is_sowordsmen,is_erotic,is_best,is_worst
0,"['邓超', '段奕宏', '郭涛', '王珞丹', '吕颂贤', '高虎', '白柳汐',...","['2015-08-27(中国大陆)', '2015-06-15(上海电影节)']",['曹保平'],24719063,['汉语普通话'],7.9,290209.0,['中国大陆'],139.0,烈日灼心,...,0,0,0,0,0,0,0,0,0,0
1,"['李易峰', '迈克尔·道格拉斯', '周冬雨', '曹炳琨', '王戈', '苏可']",['2018-06-29(中国大陆)'],['韩延'],26925317,"['汉语普通话 ', ' 英语']",7.3,209406.0,['中国大陆'],132.0,动物世界,...,0,0,0,0,1,0,0,0,0,0
2,"['尊龙', '陈冲', '邬君梅', '彼得·奥图尔', '英若诚', '吴涛', '黄自...","['1987-10-04(东京国际电影节)', '1987-10-23(意大利)']",['贝纳尔多·贝托鲁奇'],1293172,"['英语 ', ' 汉语普通话 ', ' 日语 ', ' 俄语']",9.0,222843.0,"['英国', '意大利', '中国大陆', '法国', '美国']",163.0,末代皇帝 The Last Emperor,...,1,0,0,0,0,0,0,0,1,0
3,"['周迅', '李冰冰', '张涵予', '王志文', '黄晓明', '苏有朋', '英达'...",['2009-09-29(中国大陆)'],"['陈国富', '高群书']",3734112,"['汉语普通话 ', ' 日语']",8.1,304278.0,['中国大陆'],118.0,风声,...,0,1,0,0,0,0,0,0,0,0
4,"['邓超', '彭于晏', '赵丽颖', '董子健', '金士杰', '易小星', '张本煜...",['2017-01-28(中国大陆)'],['韩寒'],26862259,['汉语普通话'],6.8,353277.0,['中国大陆'],102.0,乘风破浪,...,0,0,0,0,0,0,0,0,0,0


# Artical

# 豆瓣华语电影分析

## 1. 前言

随着生活的水平提高，人们开始了对精神生活的追求，电影已经成为我们生活中必不可少的一项娱乐活动。近年来，中国电影产业发展迅速，华语电影也引起了不少的关注。本文通过对豆瓣华语电影数据的分析，带大家了解一下华语电影这些年来的发展情况。



## 2. 数据获取

使用python的Scrapy框架爬取了豆瓣华语电影，即中国大陆、香港、台湾地区总共33133部，并将数据存储在本地的MongoDB数据库中。对于每部电影，收集以下12个字段：

* id: 豆瓣id
* title: 电影名称
* year: 上映年份
* region: 制片地区
* language: 语言
* director: 导演
* type: 类型
* actor: 主演
* date: 上映日期
* runtime: 片长
* rate: 评分
* rating_num: 参与评分人数

具体示例如下：

![数据描述](image/数据描述.png)



## 3. 数据清洗 

在进入分析之前，我们要对获取的数据进行清洗和处理。首先，使用pymongo模块将数据库中的数据导入，接着使用pandas库进行处理。

### 3.1 缺失值处理

通过打印每一列的空值统计，可以看到部分列都存在空值。之前在数据爬取的过程中，发现一般如果电影是在院线上映的且年代不是特别久远的，所有的信息都很完整。如果出现导演、演员、上映日期等的一些列为空的，大部分是网络电影或者是没有上映的。所以这里将在演员、导演、语言、类型、上映时间这5列中任一列出现空值的数据都做删除处理。另外对于年份一列有缺失的，采用上映日期的年份进行填充。

### 3.2 数据格式处理

为了后续分析方便，将部分列的数据格式进行转换，如将评分和评分人数这两列由string类型分别转为float类型和int类型；将语言一列list中的"/"分割改为","分割等。

### 3.3 增加辅助列

通过观察数据发现，如果一部电影是多个地区合作制作的，那么地区这一列的list中就有多个地区的元素。以这种展现形式，不便于后续以地区为维度进行分析。因为本篇文章主要分析华语地区，即中国大陆、香港、台湾，于是增加三列辅助列，属于该地区则标记为1，否则为0。如果这部电影的制作地区同时出现中国大陆、香港、台湾，则这三列辅助列都标记为1，表示同时属于这三个地区。对电影类型一列，也做类似的处理。



## 4. 数据分析

数据清洗处理完毕，下面就可以对数据进行分析啦。下文的画图工具，采用plotly这个作图神器，它可以使用简单的代码实现较复杂的功能。

### 4.1 行业产量和质量分析

#### 4.1.1 总体*发展趋势*

*虽然今天的我们到电影院看电影已经是十分平常的事情了，可是对于上个世纪可能还是属于一项相对“奢侈”的活动，那么我们就首先从电影行业的整体发展情况来分析一下近些年来的趋势。*

![华语电影年产量趋势图](image/华语电影年产量趋势.png)

从图中可以看到，整个华语地区，在1947年之前每年电影产量只有零星几部；从上世纪60年代开始到21世纪00年代，处于平缓发展的阶段；从2010年开始，华语电影发展迅猛，2017年年产量达到1730部，约为2010年的5.3倍。（注：因为此数据截止到2018年10月初，所以2018年的数据还有缺失。）

下面我们由每年电影的均分来看下，在数量增加的同时，质量是否有提升呢？

![华语电影评分趋势](image/华语电影评分趋势.png)

从图中看出，近些年来华语电影评分总体上呈现下降的趋势，到2016年开始才略有回升。那到底是什么导致了这个评分趋势呢？接下来，我们画个箱体图来看下每年评分的分布情况。

![华语电影评分箱体图趋势](image/华语电影评分箱体图趋势.png)

可以看到之前电影的分数段都比较集中，而近些年评分分布越来越广，从2分到接近10分的电影都有。在箱体图中，q1（图中每个长条形的下方）代表的是第一四分位，即有25%的电影评分小于此值。同理，q3（图中每个长条形的上方）有75%的电影评分小于此值。通过观察可以发现，q1、q3的值同均值的趋势一致，近些年呈现下降的趋势。这也就代表了，近年来高分片数量减少而烂片的比例增加，导致均分的下降。而此状况的好转，是2016年均分开始有所回升的原因。

*结论：*近些年人们生活水平提高了，更注重精神方面的需求，但同时也对其质量的要求也越来越高。



#### 4.1.2 地区对比

*华语电影主要有大陆、香港、台湾这几个生成地区，那么接下来，我们来对比这些地区的评分情况。*

![不同地区电影年产量对比](image/不同地区电影年产量对比.png)

可以看到，香港电影是华语电影的先驱者，20世纪整个华语电影几乎是由香港主导的。但进入2000年后，香港电影的产量下降，与此同时，大陆电影发展迅速，与2005年开始反超香港地区，替代其成为华语电影新的主导者。而台湾地区，除了六七十年代有一波发展之外，几乎处于没落、停滞不前的状态。

![不同地区电影评分趋势对比](image/不同地区电影评分趋势对比.png)

可以看到虽然台湾的产量较少，但从1983年以来，平均质量优于其他地区。而印象里，八九十年代的香港电影还不错，但从图中观察到，那时候香港电影的均值低于总体水平。反而是大陆地区的电影在那个年代整体质量比较高。

*结论：可以看出大陆电影在这些年的发展比较迅猛，但是在电影市场如此繁荣的今天，大陆电影的质量却越来越低，这或许是和这些年发展的过快而没有注重质量有关系，相比之下台湾和香港电影虽然产量较少，但是整体的质量在三个地区中都比较高，也许和台湾香港电影比较成熟有关系。*

### 4.2 影片类型分析

*电影的题材有很多种，那么华语电影中不同题材情况是怎样的呢？我们首先分析一下不同类型的电影的数量情况。*

![不同电影类型数量](image/不同电影类型数量.png)

从图中可以看到，剧情、喜剧、动作、爱情这几类的电影数量远超其他类型，其中剧情类最多，约为第二名喜剧类的1.85倍。而数量最少的类型是情色、灾难、西部片。那么哪种类型的电影关注度最多呢？

![不同类型电影关注度](image/不同类型电影关注度.png)

电影的评分人数越多，说明此电影看过的人数越多，可以从侧面反映电影的关注度。所以我们用平均评分人数的指标，来看不同类型电影的关注度。数量最少的西部片竟然是排名第一的，为什么关注度会这么高呢？华语有哪些电影是属于西部片呢？

![华语西部片](/Users/yeungshikam/Documents/数据分析项目/豆瓣华语电影分析/image/华语西部片.png)

原来是让子弹飞和无人区这两部电影拉高了西部片的平均评分人数。另外，让子弹飞是在本次爬取的全部电影中关注度Top2的，第一名为霸王别姬，一共有848429条评分。

接着，我们来看一下不同电影类型的质量。

![不同电影质量](image/不同电影质量.png)

西部、传记、歌舞这三类质量为Top3，且这三者间差距都在0.1分以内。而排名垫底的是情色、惊悚、悬疑，而这几类关注度并不是很高。评分人数与评分之间会不会有什么关系呢？

![不同类型电影评价人数与评分的关系](image/不同类型电影评分人数与评分的关系.png)

似乎没发现什么值得关注的结果，如果不以类别的平均值，而是以每部电影为维度呢？在这里我们将评分人数小于500的电影剔除，避免一些因为评分人少而出现的偏颇情况。

![评分人数与评分的关系](image/评分人数与评分的关系.png)

从图中可以看到，评分较低的，关注人数也较少。如评分小于5分，除少数几部外，其他的评分人数是在200k以下。而评分很高的电影，并不是都有很高的关注度，也有一些没什么人关注的。

*结论：*


### 4.3 导演、演员分析

这一部分，我们来看一下哪个导演或者演员属于好片或者烂片的专业户。这里好片的定义为评分大于等于8.5分，烂片为评分小于6分的电影。另外，此部分分析只针对执导或主演5部以上电影的导演和演员。

![好片占比排名前10的导演](image/好片占比排名前10的导演.png)

图中为好片占比Top10的导演，排名第一的是台湾导演杨德昌，其一共执导过8部电影，其中5部评分在8.5分以上，好片占比为62%，他的五部好片分别是《一一》、《独立时代》、《牯岭街少年杀人事件》、《恐怖分子》以及《麻将》。另外，图中除魏德圣、桑弧这两位导演各有一部烂片外，其他的导演均没有执导过6分以下的电影。

![烂片占比排名前10的导演](image/烂片占比排名前10的导演.png)

接着我们来看一下烂片占比排名10的导演，郑成峰、姜国民、管晓杰、黄柏基这四位导演的烂片比例达到了100%！

![好片占比排名前10的演员](image/好片占比排名前10的演员.png)

而在演员中，台湾演员文英的好片占比最高，为50%。值得注意的是在这10位演员中，台湾和内地演员各占50%，台湾演员除澎恰恰外，其他几位近10年内都有作品；而内地演员不是配音演员，就是活跃在上个世纪的已经息影或离世的演员。

![烂片占100%的演员](image/烂片占比前10的演员.png)

烂片占比前10的演员榜中，除邓家佳外，其他演员的烂片占比都为100%。其中大部分为内地新生代且小透明的演员。

![部分演员好烂片分布情况](image/部分演员好烂片分布情况.png)

可能大家在刚刚的分析中都没看到自己比较熟悉的演员，这里将一些比较有名的演员挑出来。从图中可以看到，部分演员参演作品的数量惊人，但好片数量却只有几部；也有几位演员的演技是大家都比较认可的，但可能挑电影的眼光不太好，其参演的作品风评都比较一般。

*结论：*

## 5. 后记

数据集中其实还有很多可以分析或者更深入的地方，但因篇幅有限，只能先挑一部分。大家看完后，如果有什么想要了解的，可以自行下载数据集进行分析，说不定还有一些有意思的小发现。

最后，给大家放送一些福利。

**福利一：9分以上且评价人数大于20万的一定不要错过的华语神片！**

![华语神片推荐](image/华语神片推荐.png)



**福利二：6分以下且评分人数超过20万的很多人看了觉得浪费生命的华语烂片！**

![不要看的华语烂片](image/不要看的华语烂片.png)